In [41]:
import os

from arcgis.gis import GIS
from arcgis.network import ClosestFacilityLayer
from arcgis.features import GeoAccessor
from arcgis.geometry import Geometry, find_transformation

In [38]:
test_gdb = os.path.abspath('../data/test/test.gdb')
store_fc = os.path.join(test_gdb, 'four_coffee')
block_fc = os.path.join(test_gdb, 'four_coffee_blocks_halfmile')
blockgroups_fc = os.path.join(test_gdb, 'four_coffee_blockgroups_halfmile')

target_sdf = GeoAccessor.from_featureclass(store_fc)
origin_sdf = GeoAccessor.from_featureclass(blockgroups_fc)
origin_id_fld = 'ID'
centroid_weighting_sdf = GeoAccessor.from_featureclass(block_fc)

In [45]:
if origin_sdf.spatial.sr != centroid_weighting_sdf.spatial.sr:
    transformation_lst = find_transformation(centroid_weighting_sdf.spatial.sr, origin_sdf.spatial.sr)
    if len(transformation_lst):
        centroid_weighting_sdf.spatial.project(origin.spatial.sr, transformation_lst[0])
    else:
        centroid_weighting_sdf.spatial.project(origin.spatial.sr)
    
transformation_lst = find_transformation(centroid_weighting_sdf.spatial.sr, origin_sdf.spatial.sr)
transformation

{'transformations': []}

In [36]:
# convert the origin areas to a spatially enabled dataframe to work wih
in_origin_sdf = origin_areas_fs.sdf

# get the geometry of the input origins
in_origin_geom = in_origin_sdf.spatial.geometry_type[0]

# if if points and and a centroid weighting features are not provided, convert them to points at the centroid
if in_origin_geom != 'point' and centroid_weighting_fs is None:
    
    print('geometric centroid')
    
    origin_sdf = in_origin_sdf[origin_areas_id_fld].to_frame()
    origin_sdf['SHAPE'] = in_origin_sdf['SHAPE'].apply(
        lambda geom: Geometry(
            {'x': geom.centroid[0], 'y': geom.centroid[1], 'spatialReference': in_origin_sdf.spatial.sr}))
    origin_sdf.spatial.set_geometry('SHAPE')

# if the geometry is not a point or multipoint, and a centroid weighting feature class is provided
elif in_origin_geom != 'point' and centroid_weighting_fs is not None:
    
    # convert the weighting feature set to a spatially enabled dataframe
    wgt_sdf = centroid_weighting_fs.sdf

    # ensure the geometry type is a point, and if not, make it points
    wgt_geom_type = wgt_sdf.spatial.geometry_type[0]
    if wgt_geom_type != 'point':
        wgt_sdf = in_origin_sdf['SHAPE'].apply(lambda geom: Geometry(
                {'x': geom.centroid[0], 'y': geom.centroid[1], 'spatialReference': in_origin_sdf.spatial.sr}
        )).to_frame()
        wgt_sdf.spatial.set_geometry('SHAPE')

    # index the weighting points to speed up the process
    wgt_sdf.spatial.sindex()

    # perform a spatial join between the weighted centroid point features and the origin areas feature classes
    wgt_joined_sdf = wgt_sdf.spatial.join(in_sdf[[origin_areas_id_fld, 'SHAPE']], how='inner', op='within')

    # pull out the coordinates into discrete fields and calculate the average by the containing origin area IDs
    wgt_joined_sdf['x'] = wgt_joined_sdf['SHAPE'].apply(lambda geom: geom.x)
    wgt_joined_sdf['y'] = wgt_joined_sdf['SHAPE'].apply(lambda geom: geom.y)
    origin_sdf = wgt_joined_sdf[[origin_areas_id_fld, 'x', 'y']].groupby(origin_areas_id_fld).mean()

    # build point geometries from the weighted centroid coordinate columns, and then drop them since no longer needed
    origin_sdf['SHAPE'] = origin_sdf.apply(lambda row: Geometry({'x': row['x'], 'y': row['y'], 'spatialReference': wgt_sdf.spatial.sr}), axis=1)
    origin_sdf.drop(columns=['x', 'y'], inplace=True)

# otherwise, if points and and a centroid weighting features are not provided, simply consolidate the dataframe
else:
    origin_sdf = in_origin_sdf[[origin_areas_id_fld, 'SHAPE']].copy()

origin_sdf.spatial.plot()

weighting


MapView(layout=Layout(height='400px', width='100%'))

In [60]:
# convert the weighting feature set to a spatially enabled dataframe
wgt_sdf = centroid_weighting_fs.sdf

# ensure the geometry type is a point, and if not, make it points
wgt_geom_type = wgt_sdf.spatial.geometry_type[0]
if wgt_geom_type != 'point':
    wgt_sdf = wgt_sdf['SHAPE'].apply(lambda geom: Geometry(
            {'x': geom.centroid[0], 'y': geom.centroid[1], 'spatialReference': wgt_sdf.spatial.sr}
    )).to_frame()
    wgt_sdf.spatial.set_geometry('SHAPE')

# index the weighting points to speed up the process
wgt_sdf.spatial.sindex()

wgt_sdf.spatial.plot()

MapView(layout=Layout(height='400px', width='100%'))

In [68]:
origin_id_fld = None
# perform a spatial join between the weighted centroid point features and the origin areas feature classes
if origin_id_fld is None:
    wgt_joined_sdf = wgt_sdf.spatial.join(in_sdf[['SHAPE']], how='inner', op='within', right_tag='origin')
    origin_id_fld = 'index_origin'
else:
    wgt_joined_sdf = wgt_sdf.spatial.join(in_sdf[[origin_id_fld, 'SHAPE']], how='inner', op='within')

# pull out the coordinates into discrete fields and calculate the average by the containing origin area IDs
wgt_joined_sdf['x'] = wgt_joined_sdf['SHAPE'].apply(lambda geom: geom.x)
wgt_joined_sdf['y'] = wgt_joined_sdf['SHAPE'].apply(lambda geom: geom.y)
cent_df = wgt_joined_sdf[[origin_id_fld, 'x', 'y']].groupby(origin_id_fld).mean()

# build point geometries from the weighted centroid coordinate columns, and then drop them since no longer needed
cent_df['SHAPE'] = cent_df.apply(lambda row: Geometry({'x': row['x'], 'y': row['y'], 'spatialReference': wgt_sdf.spatial.sr}), axis=1)
cent_df = cent_df.drop(columns=['x', 'y']).reset_index()
cent_df.sample(5)

,index_origin,SHAPE
14,14,"{'x': -123.04959354094972, 'y': 44.96014914452..."
0,0,"{'x': -123.03360475180584, 'y': 44.95077184665..."
1,1,"{'x': -123.03781638931511, 'y': 44.94302082185..."
3,3,"{'x': -123.01572241568535, 'y': 44.94673090803..."
9,9,"{'x': -123.02291115428436, 'y': 44.92539807103..."


In [67]:
gis = GIS(username='jmccune_geoai')

Enter password:  ··········


In [69]:
closest_facility_lyr = ClosestFacilityLayer(gis.properties.helperServices.closestFacility.url, gis)
closest_facility_lyr

<ClosestFacilityLayer url:"https://route.arcgis.com/arcgis/rest/services/World/ClosestFacility/NAServer/ClosestFacility_World">

In [70]:
closest_facility_lyr.properties

{
  "currentVersion": 10.7,
  "layerName": "ClosestFacility_World",
  "layerType": "esriNAServerClosestFacilityLayer",
  "impedance": "TravelTime",
  "restrictions": [
    "Avoid Unpaved Roads",
    "Avoid Private Roads",
    "Driving an Automobile",
    "Through Traffic Prohibited",
    "Roads Under Construction Prohibited",
    "Avoid Gates",
    "Avoid Express Lanes",
    "Avoid Carpool Roads"
  ],
  "snapTolerance": 0,
  "maxSnapTolerance": 20000,
  "snapToleranceUnits": "esriMeters",
  "ignoreInvalidLocations": true,
  "restrictUTurns": "esriNFSBAtDeadEndsAndIntersections",
  "accumulateAttributeNames": [
    "Miles",
    "Kilometers"
  ],
  "attributeParameterValues": [
    {
      "attributeName": "Avoid Limited Access Roads",
      "parameterName": "Restriction Usage",
      "parameterType": "float",
      "value": "Avoid_Medium"
    },
    {
      "attributeName": "Avoid Ferries",
      "parameterName": "Restriction Usage",
      "parameterType": "float",
      "value": "Avoid